In [1]:
%load_ext autoreload
%autoreload 2
import os

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"  # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = "MIG-864c07c4-8eeb-5b23-8d57-eaeb942a9a0f"
import matplotlib.pyplot as plt
import numpy as np
import torch
import yaml
from hydra.utils import instantiate
from PIL import Image
from torch.utils.data import DataLoader, Dataset
import pandas as pd
from br.models.load_models import get_data_and_models
from br.models.save_embeddings import get_pc_loss, save_embeddings, save_emissions, get_pc_loss_chamfer
from br.models.compute_features import compute_features
from br.models.utils import get_all_configs_per_dataset
from br.features.plot import collect_outputs, plot, plot_stratified_pc
from br.models.compute_features import get_embeddings
from br.models.utils import get_all_configs_per_dataset
from br.features.reconstruction import stratified_latent_walk
from br.features.archetype import AA_Fast
from pathlib import Path
import pandas as pd
from br.features.utils import normalize_intensities_and_get_colormap, normalize_intensities_and_get_colormap_apply
import os
device = "cuda:0"

In [2]:
os.chdir("/allen/aics/modeling/ritvik/projects/benchmarking_representations/")
save_path = "./test_npm1_save_embeddings/"

In [11]:
dataset_name = "npm1"
batch_size = 2
debug = False
results_path = (
    "/allen/aics/modeling/ritvik/projects/benchmarking_representations/configs/results/"
)
data_list, all_models, run_names, model_sizes = get_data_and_models(
    dataset_name, batch_size, results_path, debug
)

# Compute embeddings and emissions

In [13]:
run_names

['SO3_pointcloud_SDF',
 'SO3_image_SDF',
 'SO3_image_seg',
 'Classical_image_SDF',
 'Classical_image_seg']

In [ ]:
from br.models.save_embeddings import save_embeddings

splits_list = ["test"]
meta_key = None
eval_scaled_img = [True] * 5

gt_mesh_dir = "/allen/aics/assay-dev/users/Alex/replearn/rep_paper/data/var_blobby_noalign/meshes"
gt_sampled_pts_dir = "/allen/aics/assay-dev/users/Alex/replearn/rep_paper/data/sampled_pcs/npm1_var_noalign_global/1_res/0"
gt_scale_factor_dict_path = "/allen/aics/assay-dev/users/Alex/replearn/rep_paper/data/npm1_var_scale_factor_32res_noalign_global.npz"

eval_scaled_img_params = [
                        {"eval_scaled_img_model_type":"iae",
                          "eval_scaled_img_resolution":32,
                          "gt_mesh_dir":gt_mesh_dir,
                          "gt_scale_factor_dict_path":None,
                          "gt_sampled_pts_dir":gt_sampled_pts_dir,
                          "mesh_ext":"stl"},
                        {"eval_scaled_img_model_type":"sdf",
                          "eval_scaled_img_resolution":32,
                          "gt_mesh_dir":gt_mesh_dir,
                          "gt_scale_factor_dict_path":gt_scale_factor_dict_path,
                          "gt_sampled_pts_dir":None,
                          "mesh_ext":"stl"},
                        {"eval_scaled_img_model_type":"seg",
                          "eval_scaled_img_resolution":32,
                          "gt_mesh_dir":gt_mesh_dir,
                          "gt_scale_factor_dict_path":gt_scale_factor_dict_path,
                          "gt_sampled_pts_dir":None,
                          "mesh_ext":"stl"},
                        {"eval_scaled_img_model_type":"sdf",
                          "eval_scaled_img_resolution":32,
                          "gt_mesh_dir":gt_mesh_dir,
                          "gt_scale_factor_dict_path":gt_scale_factor_dict_path,
                          "gt_sampled_pts_dir":None,
                          "mesh_ext":"stl"},
                        {"eval_scaled_img_model_type":"seg",
                          "eval_scaled_img_resolution":32,
                          "gt_mesh_dir":gt_mesh_dir,
                          "gt_scale_factor_dict_path":gt_scale_factor_dict_path,
                          "gt_sampled_pts_dir":None,
                          "mesh_ext":"stl"},]
loss_eval_list = [torch.nn.MSELoss(reduction='none')]*5
sample_points_list = [False]*5
skew_scale = None
save_embeddings(
    save_path,
    data_list,
    all_models,
    run_names,
    debug,
    splits_list,
    device,
    meta_key,
    loss_eval_list,
    sample_points_list,
    skew_scale,
    eval_scaled_img,
    eval_scaled_img_params,
)

Processing test


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 887/887 [48:37<00:00,  3.29s/it]


Processing test


  0%|▎                                                                                                               | 2/887 [00:05<42:07,  2.86s/it]

Created empty mesh


...

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 887/887 [39:22<00:00,  2.66s/it]


Processing test


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 887/887 [38:18<00:00,  2.59s/it]


Processing test


 13%|██████████████▏                                                                                               | 114/887 [04:43<30:34,  2.37s/it]

In [ ]:
this